<a href="https://colab.research.google.com/github/jhk0530/spider/blob/master/43_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EB%82%B4%EB%B6%80%EC%9D%98_%EB%AC%B8%EC%9E%90_%EC%9D%B8%EC%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 이미지 데이터 다운로드 받기. 

In [1]:
URL = 'http://yann.lecun.com/exdb/mnist/'


import urllib.request as req
import gzip, os, os.path

savepath = './mnist'
baseurl = URL

files = [
         "train-images-idx3-ubyte.gz",
         "train-labels-idx1-ubyte.gz",
         "t10k-images-idx3-ubyte.gz",
         "t10k-labels-idx1-ubyte.gz"
]

if not os.path.exists(savepath): os.mkdir(savepath)

for f in files:
  url = baseurl + '/' + f
  loc = savepath + '/' + f
  print('download : ' , url)
  if not os.path.exists(loc):
    req.urlretrieve(url,loc)

for f in files:
  gz_file = savepath + '/' + f
  raw_file = savepath + '/' + f.replace('.gz', '')
  print('gzip:', f)
  with gzip.open(gz_file,'rb') as fp:
    body = fp.read()
    with open(raw_file, 'wb') as w:
      w.write(body)

print("ok")

download :  http://yann.lecun.com/exdb/mnist//train-images-idx3-ubyte.gz
download :  http://yann.lecun.com/exdb/mnist//train-labels-idx1-ubyte.gz
download :  http://yann.lecun.com/exdb/mnist//t10k-images-idx3-ubyte.gz
download :  http://yann.lecun.com/exdb/mnist//t10k-labels-idx1-ubyte.gz
gzip: train-images-idx3-ubyte.gz
gzip: train-labels-idx1-ubyte.gz
gzip: t10k-images-idx3-ubyte.gz
gzip: t10k-labels-idx1-ubyte.gz
ok


## 이미지 데이터 변환하기 (binary to csv)

* `~ubyte` 는 자체적인 db형식이라서 로드 불가능.

바이너리 데이터는 다루기 어렵기 때문에 csv로 변환.

목표 : 

`<label>` `<28*28 개 짜리 pixel data>` <br>
* ex) 5, 0,0,0,0,.....0 ( 28*28 ) 개








In [0]:
import struct

def to_csv(name, maxdata):
  lbl_f = open('./mnist/'+name+'-labels-idx1-ubyte','rb')
  img_f = open('./mnist/'+name+'-images-idx3-ubyte','rb')
  csv_f = open('./mnist/'+name+'.csv','w', encoding = 'utf-8')

  mag, lbl_count = struct.unpack(">II", lbl_f.read(8))
  mag, img_count = struct.unpack(">II", img_f.read(8))
  rows, cols = struct.unpack('>II', img_f.read(8))
  pixels = rows * cols

  res = []

  for idx in range(lbl_count):
    if idx > maxdata : break
    label = struct.unpack("B", lbl_f.read(1))[0]
    bdata = img_f.read(pixels)
    sdata = list(map(lambda n : str(n), bdata))
    csv_f.write(str(label)+',')
    csv_f.write(','.join(sdata) + '\r\n')

    if idx < 10 : 
      s= "P2 28 28 255\n"
      s += ' '.join(sdata)
      iname = './mnist/{0}-{1}-{2}.pgm'.format(name,idx,label)
      with open(iname, 'w', encoding = 'utf-8') as f:
        f.write(s)
  
  csv_f.close()
  lbl_f.close()
  img_f.close()
  
to_csv('train' ,1000)
to_csv('t10k' ,1000)


## 이미지 데이터 학습 시키기

이미지를 벡터로 변환 시켜서 입력후 테스트.



In [9]:
from sklearn import model_selection, svm, metrics

def load_csv(fname):
  labels = []
  images =[]
  
  with open(fname , 'r') as f:
    for line in f:
      cols = line.split(',')
      if len(cols)<2: continue
      labels.append(int(cols.pop(0)))
      vals = list(map(lambda n: int(n)/ 256, cols))
      images.append(vals)
  return {'labels':labels, "images": images}

data = load_csv('./mnist/train.csv')
test = load_csv('./mnist/t10k.csv')

clf = svm.SVC()
clf.fit(data['images'], data['labels'])

predict = clf.predict(test['images'])

ac_score = metrics.accuracy_score(test['labels'], predict)
cl_report = metrics.classification_report(test['labels'], predict)

print('정답률 = ', ac_score)
print('리포트 =')
print(cl_report)

정답률 =  0.8751248751248751
리포트 =
              precision    recall  f1-score   support

           0       0.91      0.95      0.93        85
           1       0.98      0.98      0.98       126
           2       0.91      0.88      0.89       116
           3       0.94      0.75      0.83       107
           4       0.85      0.86      0.86       110
           5       0.73      0.86      0.79        87
           6       0.92      0.87      0.89        87
           7       0.84      0.90      0.87        99
           8       0.86      0.78      0.82        89
           9       0.82      0.89      0.85        95

    accuracy                           0.88      1001
   macro avg       0.87      0.87      0.87      1001
weighted avg       0.88      0.88      0.88      1001

